In [1]:
1+1

2

In [3]:
a = 1
b = 2
c = a + b
print(c)

3


In [4]:
### Introduction to the data ingestion module


In [5]:
import os
from typing import List, Dict, Any
import pandas as pd
import json


In [10]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter)
print("Data ingestion module loaded successfully.")

Data ingestion module loaded successfully.


## understand the document structure

In [16]:
##create a simple document
doc = Document(page_content="This is a sample document for data ingestion.", 
               metadata={"source": "sample.txt",
                         "page": 1,
                         "title": "Sample Document",
                         "author": "John Doe" })
print(f"Document Structure:")
print(doc.page_content)
print(json.dumps(doc.metadata))
print("\n Metadata is crucial for filtering and retrieval in RAG systems.")
print("Tracking document source")
print("providing context in responses")
print("debuging and auditing")

Document Structure:
This is a sample document for data ingestion.
{"source": "sample.txt", "page": 1, "title": "Sample Document", "author": "John Doe"}

 Metadata is crucial for filtering and retrieval in RAG systems.
Tracking document source
providing context in responses
debuging and auditing
